# TIM - TP 7 : Détections de coins

Aujourd'hui, on va s'intéresser à plusieurs algorithmes permettant de faire de la détection de coins. Un coin est une catégorie de points d'intérêts, c'est-à-dire un point d'une image avec une caractéristique spécifique, qui peut ensuite être utiliser dans plusieurs tâches, comme du pattern matching. 

In [ ]:
# A COMPLETER
# Chargement des librairies
...

# Définition du chemin de la base de données des images
...

## Ex. 1 : Détecteur de coins d'Harris

Commençons par un classique : le détecteur de coin d'Harris. Cet algorithme permet de détecter donc des coins, à savoir des grandes variations d'intensités locales dans toutes les directions. 

---
#### Description mathématique de l'opérateur

L'idée est de considérer un voisinage centré autour d'un pixel, de le décaler légèrement dans plusieurs directions, et de calculer pour chaque déplacement la variation d'intensité. Cela se traduit mathématiquement par la fonction suivante :

\begin{equation*}
    E(u,v) = \sum_{x,y} W(x,y) \cdot [I(x+u,y+v) - I(x,y)]²
\end{equation*}

Dans cette équation:
- $I(x,y)$ correspond à l'intensité de l'image
- $(u,v)$ est un petit déplacement
- $W(x,y)$ est une fenêtre gaussienne locale centrée en $(x,y)$

$E(u,v)$ représente donc la différence d'intensité de $I$ dans une direction $(u,v)$. De ce fait:
- Si $E(u,v)$ est faible dans toutes les directions, la région est plate
- Si $E(u,v)$ est fort dans une seule direction, on a un bord
- Si $E(u,v)$ est fort dans toutes les directions, on a un coin

Grâce à une approximation du premier ordre de Taylor, on peut linéariser $I(x+u,y+v)$ pour des petits déplacements: 

\begin{equation*}
    I(x+u,y+v) \approx I(x,y) + I_x(x,y) \cdot u + I_y(x,y) \cdot v
\end{equation*}

Et donc :

\begin{equation*}
    E(u,v) \approx \sum_{x,y} W(x,y) \cdot [I_x u - I_y v]²
\end{equation*}

En développant le carré : 

\begin{equation*}
    E(u,v) \approx \sum_{x,y} W(x,y) \cdot (I_x² u² + 2 I_x I_y u v + I_y² v²)
\end{equation*}

On écrit sous forme matricielle :

\begin{equation*}
    E(u,v) \approx \begin{bmatrix} u & v \end{bmatrix} M \begin{bmatrix} u \\ v \end{bmatrix}
\end{equation*}

Avec $M$ la matrice des moments du gradient:

\begin{equation*}
    M(x,y) =  \sum_{x,y} W(x,y)  \begin{bmatrix}
                I_{x}² & I_x I_y \\
                I_x I_y & I_{y}²
                \end{bmatrix}
\end{equation*}

Maintenant, interprétons $M$. C'est une matrice symétrique $2 \times 2$, donc elle a deux valeurs propres réelles positives $\lambda_1$ et $\lambda_2$. On peut donc réecrire l'équation de cette manière : 

\begin{equation*}
    E(u,v) \approx \lambda_1 a_1² + \lambda_2 a_2²
\end{equation*}

Où a_1 et a_2 sont les composantes de $(u,v)$ dans la base propre de $M$. De ce fait :
- Si les deux $\lambda$ sont petits, $E(u,v)$ varie peu et donc on est en région plate
- Si un seul $\lambda$ est grand, $E(u,v)$ varie dans une seule direction, et donc on est sur un bord
- Si les deux $\lambda$ sont grands, $E(u,v)$ varie dans toutes les directions, et donc on a un coin.

Cependant, calculer $\lambda_1$ et $\lambda_2$ sur toute l'image est coûteux. Harris propose donc pour cela un critère scalaire approché, appelé critère de Harris :

\begin{equation*}
    R = det(M) - k(trace(M))²
\end{equation*}

Avec :

\begin{equation*}
    \begin{cases}
        det(M) = \lambda_1 \lambda_2 \\
        trace(M) = \lambda_1 + \lambda_2
    \end{cases}
\end{equation*}

Et $k$ est un paramètre empirique (souvent entre 0.04 et 0.06). En remplaçant les $\lambda$ par les termes de $M$, on obtient l'équation finale du critère de Harris :

\begin{equation*}
    R = (S_{xx} S_{yy} - S_{xy}²) - k (S_{xx} + S_{yy})²
\end{equation*}

Avec $S_{xx} = G \ast I_x²$, $S_{xy} = G \ast (I_x I_y)$ et $S_{yy} = G \ast I_y²$. $G$ correspond à un filtre gaussien pour lisser les produits de gradients, afin de stabiliser la détection face au bruit et donner plus de poids au centre de la fenêtre.

---

On va recoder l'algorithme en entier, même s'il existe évidemment une implémentation d'OpenCV. Commençons d'abord par charger l'image exemple du jour : *blox.jpg*

In [ ]:
# A COMPLETER
# Chargement de l'image et affichage
...

La première étape consiste à lisser l'image pour éviter que le bruit local puisse être considéré comme un contour. Appliquez donc un filtre gaussien 3x3 sur l'image.

In [ ]:
# A COMPLETER
# Filtrage Gaussien 3X3 pour lisser l'image
...

La détection de coins se basent sur des contours. Pour cela, Harris se base sur les gradients directionnels $I_x$ et $I_y$. Il y'a plusieurs possibilités ici, mais disons que les filtres de Sobel sont les plus couramment utilisés... Donc on va faire pareil. 

Extrayez les gradients directionnels $I_x$ et $I_y$ via des filtres de Sobel.

In [ ]:
# A COMPLETER
# Extraction et affichage des gradients directionnels Ix et Iy via des filtres de Sobel
...

On va maintenant calculer comme expliqué précédemment $S_{xx}$, $S_{xy}$ et $S_{yy}$. Le noyau gaussien à appliquer $G$ est de taille $blocksize \times blocksize$ (variable qui sera en paramètre de l'algorithme).

Construisez $S_{xx}$, $S_{xy}$ et $S_{yy}$ et affichez les produits de gradients.

In [ ]:
# A COMPLETER
# Construction et affichage de Sxx, Sxy et Syy
...

On va maintenant calculer directement le critère de Harris :

\begin{equation*}
    R(x,y) = (S_{xx} S_{yy} - S_{xy}²) - k (S_{xx} + S_{yy})²
\end{equation*}

$k$ est en paramètre de la fonction.

Calculez $R(x,y)$ et affichez le résultat.

In [ ]:
# A COMPLETER
# Calcul et affichage de R(x,y)
...

**_QUESTION :_** Comment interpréter les valeurs de $R(x,y)$ (c'est-à-dire les pixels blancs/noirs/gris)? 

**_REPONSE :_**

Il est temps de sélectionner uniquement les pixels candidats à être des coins, donc les fortes valeurs positives. Pour cela, on va simplement faire une binarisation à un seuil $\tau = \alpha \cdot max(R)$. Ici, on va prendre $\alpha$ à 0.01, mais il sera un paramètre dans la fonction globale.

Faites la binarisation et visualisez le résultat.

In [ ]:
# A COMPLETER
# Binarisation pour sélectionner les pixels candidats à être des coins et affichage du résultat
...

Bon, la visualisation de pixels noirs et blancs peut ne pas être très pertinent... Créez une visualisation de l'image originale avec un cercle rouge dessiné autour de chaque pixel candidat.

*Note : votre image noir et blanc doit évidemment être convertie en couleur pour une telle visualisation*

In [ ]:
# A COMPLETER
# Visualisation des pixels candidats sur l'image originale
...

**_QUESTION :_** Que constatez-vous ?

**_REPONSE :_** 

Une dernière étape est nécessaire en post-traitement : la Non-Maximum Suppression (NMS). C'est un algorithme de post-traitement très largement utilisé (même en Deep Learning !), permettant d'éliminer les détections multiples d'un même coin dans notre cas. 

Le principe qu'on va utiliser ici est ce qu'on appelle un Greedy NMS. On récupère tous les coordonnées des pixels candidats $(x_c, y_c)$ et leurs scores $s_c = R(x_c,y_c)$. On trie ensuite chaque pixel candidat $(x_c, y_c, s_c)$ dans l'ordre décroissant de score, considérant qu'un pixel est plus probablement un coin si son score est gros. 

Puis, on crée une matrice de booléens $allow$ de la taille de $R(x,y)$ initialisé à True partout. On parcourt chacun des candidats dans la liste triée et on regarde la valeur de $allow$ aux coordonnées du pixel candidat :
- Si $R(x_c,y_c)$ = True, alors le pixel est retenu, et la fenêtre carrée de taille $(2 \times min\_distance + 1 , 2 \times min\_distance + 1)$ centrée sur $(x_c,y_c)$ de $allow$ passe à False
- Si $R(x_c,y_c)$ = False, alors le pixel est éliminé

La fonction retourne les positions $(x_c, y_c)$ des pixels retenus. $min\_distance$ est un paramètre qui sera défini dans la fonction globale.

A vous de jouer, et codez la fonction de Non-Maximum Suppression

In [ ]:
# A COMPLETER
# Non-maximum Suppression
...

Appliquez maintenant le NMS sur vos résultats précédents, et affichez l'image avec les coins détectés restants entourés en rouge.

In [ ]:
# A COMPLETER
# Post-traitement NMS et affichage des coins détectés restants
...

**_QUESTION :_** Alors, le résultat est-il meilleur ? Votre algorithme répond-il totalement aux attentes ?

**_REPONSE :_**

Regroupez tout votre code dans une fonction pour la détection de coins d'Harris. Mettez bien certaines variables en paramètres (déjà évoqué précédemment).

In [ ]:
# A COMPLETER
# Fonction de détection de coins d'Harris
...

Bravo, ce n'était pas facile, mais vous avez réussi à coder le détecteur de coins d'Harris ! On va maintenant voir comment le faire avec OpenCV. 

OpenCV propose 2 méthodes pour appliquer Harris :
- *cv2.cornerHarris*, qui renvoit la carte $R(x,y)$ comme effectué précédemment, et sans post-traitement (NMS)
- *cv2.goodFeaturesToTrack* avec l'option *useHarrisDetector=True*, qui performe Harris avec un post-traitement NMS et renvoie les coordonnées des coins.

On pourrait comparer les cartes $R(x,y)$ produites par votre algorithme et celui d'OpenCV, mais bon, on va plutôt directement utiliser *goodFeaturesToTrack*. 

Faites une détection de coins d'Harris avec cette fonction d'OpenCV et visualisez le résultat. A vous de trouver des paramètres adéquats.

In [ ]:
# A COMPLETER
# Détection de coins d'Harris via cv2.goodFeaturesToTrack
...

## Ex. 2 : Détecteurs de coins de Shi-Tomasi

Un deuxième algorithme pour la détection de coins : Shi-Tomasi. Rassurez-vous, ça va être plus rapide que pour l'exercice précédent, puisque Shi-Tomasi repose sur le même principe que Harris.

Pour rappel, avec Harris, on calculait le critère de Harris, de la forme suivante : 

\begin{equation*}
    R = det(M) - k(trace(M))²
\end{equation*}

Avec :

\begin{equation*}
    \begin{cases}
        det(M) = \lambda_1 \lambda_2 = S_{xx} + S_{yy} \\
        trace(M) = \lambda_1 + \lambda_2 = S_{xx} S_{yy} - S_{xy}²
    \end{cases}
\end{equation*}

Shi-Tomasi lui définit le score suivant :

\begin{equation*}
    score(x,y) = min(\lambda_1, \lambda_2)
\end{equation*}

Un coin est alors détecté si ce score dépasse un seuil prédéfini. 

Pour éviter une diagonalisation de $M$ pour trouver les valeurs propres, on calcule les valeurs propres à partir de $det(M)$ et $trace(M)$ :

\begin{equation*}
    \lambda_{1,2} = \frac{trace(M) \pm \sqrt{trace(M)² - 4 \cdot det(M)}}{2}
\end{equation*}

Ici, on a besoin uniquement du minimum entre $\lambda_1$ et $\lambda_2$ :

\begin{equation*}
    min(\lambda_1,\lambda_2) = \frac{trace(M) - \sqrt{trace(M)² - 4 \cdot det(M)}}{2}
\end{equation*}

De ce fait, à la place de la carte $R(x,y)$ pour Harris, Shi-Tomasi calcule la carte $\lambda_{min}$ en suivant l'équation ci-dessus (pour chaque pixel).

Modifiez la fonction de détection d'Harris pour effectuer une détection de coins de Shi-Tomasi

In [ ]:
# A COMPLETER
# Fonction de détections de coins de Shi-Tomasi
...

Appliquez votre algorithme sur l'image du jour et visualisez le résultat (avec les coins détectés entourés en rouge). Ajustez les paramètres afin d'obtenir le meilleur résultat.

In [ ]:
# A COMPLETER
# Application de Shi-Tomasi et visualisation des résultats
...

**_QUESTION :_** Obtenez-vous un meilleur résultat qu'avec le détecteur d'Harris ?

**_REPONSE :_**

Tout comme Harris, OpenCV propose deux implémentations de Shi-Tomasi :
- *cv2.cornerMinEigenVal*, qui renvoit la carte $\lambda_{min}$ sans post-processing NMS
- *cv2.goodFeaturesToTrack*, qui par défaut (*useHarrisDetector=False*), utilise Shi-Tomasi avec un post-processing NMS.

Effectuez une détection de Shi-Tomasi avec OpenCV et affichez le résultat

In [ ]:
# A COMPLETER
# Shi-Tomasi via OpenCV et visualisation du résultat
...

Pour finir avec les détections de coins, on applique souvent en post-processing un algorithme permettant d'améliorer la localisation des coins détectés. L'algorithme étant un peu complexe, on va uniquement utiliser la fonction fournie par OpenCV : *cv2.cornerSubPix()*.

Améliorez vos résultats précédants (Harris et Shi-Tomasi) en appliquant *cv2.cornerSubPix* et visualisez les résultats.

In [ ]:
# A COMPLETER
# Post-traitement cornerSubPix sur les coins détectés (Harris et Shi-Tomasi) et visualisation des résultats
...
